In [50]:
# IMPORTS
import os
import numpy as np
import time as time
import cv2


In [51]:
#DONE: # "PLT_coll4_exp.63_control" # "exp89_PRP_FBG_CONTROL01_R3D" #
file_name =   "/Users/yishaiazabary/PycharmProjects/platelets/t1"    # "PLT_col4_full"  #"exp89_PRP_FBG_TAXOL01_R3D"  #   

interval_of_interest = [64,128]
second_interval_of_interest = [64,128]
delta_of_interest = [-18,0]
second_delta_of_interest = [0.01,18]
RED = [0, 0, 255]
BLUE = [255, 0, 0]




In [52]:
def manipulate_frame(buf):
    # todo: try creating the mask from buf[0,:,:,1] so there is only one comparison and not 3, 
    # FIRST INTEREST (ATTACH)
    interval_mask_first_frame = np.where((buf[0,:,:, :1] >= interval_of_interest[0]) & (buf[0,:,:, :1]<interval_of_interest[1]))
    first_frame_vals = buf[0][interval_mask_first_frame[0], interval_mask_first_frame[1], interval_mask_first_frame[2]]
    next_frame_vals = buf[1][interval_mask_first_frame[0], interval_mask_first_frame[1], interval_mask_first_frame[2]]
    # FIRST INTEREST (DETACH)
    second_interval_mask_first_frame = np.where((buf[0] >= second_interval_of_interest[0]) & (buf[0]<second_interval_of_interest[1]))
    second_interest_first_frame_vals = buf[0][second_interval_mask_first_frame[0], second_interval_mask_first_frame[1], second_interval_mask_first_frame[2]]
    second_interest_next_frame_vals = buf[1][second_interval_mask_first_frame[0], second_interval_mask_first_frame[1], second_interval_mask_first_frame[2]]
    for k in range(len(interval_mask_first_frame[0])):
        delta_between_frames =  next_frame_vals[k] - first_frame_vals[k]  
        if delta_of_interest[0] <= delta_between_frames < delta_of_interest[1]:
            x, y, z = interval_mask_first_frame[0][k], interval_mask_first_frame[1][k], interval_mask_first_frame[2][k]
            buf[0][x][y] = np.array(RED, np.dtype('int64'))
    for j in range(len(second_interval_mask_first_frame[0])):
        delta_between_frames =  second_interest_next_frame_vals[j] - second_interest_first_frame_vals[j]  
        if second_delta_of_interest[0] <= delta_between_frames < second_delta_of_interest[1]:
            x, y, z = second_interval_mask_first_frame[0][j], second_interval_mask_first_frame[1][j], second_interval_mask_first_frame[2][j]
            buf[0][x][y] = np.array(BLUE, np.dtype('int64'))
    return buf   
                

In [53]:
total_start = time.time()
frames_in_memory = 2

cap = cv2.VideoCapture(file_name + ".avi")

final_frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = cap.get(cv2.CAP_PROP_FPS)

out = cv2.VideoWriter(file_name+'_manipulated.avi',cv2.VideoWriter_fourcc('M','J','P','G'), fps, (frame_width, frame_height))

# new_video = np.zeros((final_frame_count, frame_height, frame_width, 3), np.dtype('int64'))
buf = np.zeros((frames_in_memory, frame_height, frame_width, 3), np.dtype('int64'))
ret = True

fc = 0
while fc < final_frame_count and ret:
    # loading the data according
    in_memory_frames_ctr = 0
    single_frame_start_time = time.time()
    while in_memory_frames_ctr < frames_in_memory:
        if np.sum(buf[frames_in_memory - 1]) > 0:
            temp = buf[frames_in_memory - 1].copy()
            # buf[FRAMES_IN_MEMORY-1] = np.zeros((frame_height, frame_width, 3))
            buf = np.zeros((frames_in_memory, frame_height, frame_width, 3), np.dtype('int64'))
            buf[0] = temp
        else:
            ret, frame = cap.read()
            buf[in_memory_frames_ctr] = frame
            fc += 1
        in_memory_frames_ctr += 1
    buf = manipulate_frame(buf)
    im = np.uint8(buf[0])
    out.write(im)
    # new_video[fc-2] = buf[0]
    single_frame_end_time = time.time()
    print("frame No:{0} has been manipulated.took:{1:.2} seconds".format(fc-2, single_frame_end_time - single_frame_start_time))

out.release()
cap.release()
total_end = time.time()
print("{0}: {1:.2f} minutes total".format(fc, (total_end - total_start) / 60))
os.system('afplay alarm.m4a')

frame No:0 has been manipulated.took:0.016 seconds
frame No:1 has been manipulated.took:0.016 seconds
frame No:2 has been manipulated.took:0.0099 seconds
frame No:3 has been manipulated.took:0.0096 seconds
frame No:4 has been manipulated.took:0.0072 seconds
frame No:5 has been manipulated.took:0.01 seconds
frame No:6 has been manipulated.took:0.011 seconds
frame No:7 has been manipulated.took:0.0067 seconds
frame No:8 has been manipulated.took:0.0096 seconds
frame No:9 has been manipulated.took:0.012 seconds
frame No:10 has been manipulated.took:0.0092 seconds
frame No:11 has been manipulated.took:0.0054 seconds
frame No:12 has been manipulated.took:0.0088 seconds
frame No:13 has been manipulated.took:0.0065 seconds
frame No:14 has been manipulated.took:0.0069 seconds
frame No:15 has been manipulated.took:0.0074 seconds
frame No:16 has been manipulated.took:0.0066 seconds
frame No:17 has been manipulated.took:0.0053 seconds
frame No:18 has been manipulated.took:0.0078 seconds
frame No:

0

In [54]:
print('done')

done
